## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [6]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [7]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

### Set up the Qiskit runtime session

In [8]:
service = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="system-request/7-8-22-access/main",
    token="af0c18b11e3ac4f44582dcccc09b543c998e3caacb5bc151f059ef098195be53f1e33c5c71933bb3c1e68fc8173fc67e5433f6a2a0d7457ee875b6712c2cee1a",
)
session = Session(service=service, backend="ibmq_qasm_simulator")

### Find cuts that match our criteria

In [9]:
# Set the Sampler options
options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})

# Create the Sampler with a Qiskit Runtime session
sampler = Sampler(session=session, options=options)

# Instantiate a WireCutter and decompose the circuit
cutter = WireCutter(qc, sampler)
cuts = cutter.decompose(
    method="automatic",
    max_subcircuit_width=6,
    max_cuts=2,
    num_subcircuits=[2],
)

Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 20 rows and 5 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 24 substitutions.
Reduced MIP has 44 rows, 19 columns, and 131 nonzeros.
Reduced MIP has 15 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.31 ticks)
Found incumbent of value 1.000000 after 0.01 sec. (0.44 ticks)
Probing fixed 0 vars, tightened 2 bounds.
Probing time = 0.00 sec. (0.08 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 3 columns.
Reduced MIP has 44 rows, 16 columns, and 131 nonzeros.
Reduced MIP has 12 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Detecti

### Evaluate the subcircuits, then recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [10]:
# Evaluate the subcircuits on backend
subcircuit_instance_probabilities = cutter.evaluate(cuts)

# Recompose the circuit and generate the cut circuit's probability distribution
ordered_probabilities = cutter.recompose(
    subcircuit_instance_probabilities, cuts, num_threads=4
)

# Use a statevector simulator to calculate the error between the inferred and actual distributions
metrics = cutter.verify(ordered_probabilities)
print(metrics)

{'nearest': {'chi2': 0.0013598472403334749, 'Mean Squared Error': 2.0551891833040718e-06, 'Mean Absolute Percentage Error': 7.337723194443719, 'Cross Entropy': 2.601056707001134, 'HOP': 0.9034889042377472}, 'naive': {'chi2': 0.0013598472403334749, 'Mean Squared Error': 2.0551891833040718e-06, 'Mean Absolute Percentage Error': 7.337723194443719, 'Cross Entropy': 2.601056707001134, 'HOP': 0.9034889042377472}}
